In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import Bio
from Bio import SeqIO
from pandas import Series
import argparse
import re

In [2]:
chrlist = [''.join(['chr', str(i)]) for i in list(range(1, 23)) + ['X', 'Y']]

fa = dict()
for record in SeqIO.parse('/Users/yzli/Desktop/weilab/reference/hg38_genome.fa','fasta'):
    if record.id in chrlist:
        fa[record.id] = record

In [3]:
#forward strand sgRNA

In [4]:
nosp = pd.read_csv('../data/tyr/nosp_y_genpos.txt',sep='\t')
sp1_23 = pd.read_csv('../data/tyr/sp1_23_y_genpos.txt',sep='\t')
sp12_3 = pd.read_csv('../data/tyr/sp12_3_y_genpos.txt',sep='\t')

In [5]:
sp1_23 = sp1_23[['chr','feature','strand','gene_name','transcript_id',
               'pro_id','base1','base2','base3','aa_genome_seq','aa_position']]
sp12_3 = sp12_3[['chr','feature','strand','gene_name','transcript_id',
               'pro_id','base1','base2','base3','aa_genome_seq','aa_position']]

In [6]:
nosp.loc[:,'type'] = 'nosp'
sp1_23.loc[:,'type'] = 'sp1_23'
sp12_3.loc[:,'type'] = 'sp12_3'

In [7]:
genpos_all = nosp.append(sp1_23).append(sp12_3)

In [8]:
genpos_all.shape

(406031, 12)

In [9]:
genpos_f = genpos_all[genpos_all['strand'] == '+']
genpos_r = genpos_all[genpos_all['strand'] == '-']

In [10]:
genpos_f.loc[:,'reg_from'] = genpos_f['base2'] + 14
genpos_f.loc[:,'reg_to'] = genpos_f['base2'] + 20

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
genpos_f.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to
0,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,nosp,18400034,18400040
1,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400037,18400038,18400039,TAT,12,nosp,18400052,18400058
2,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400229,18400230,18400231,TAC,76,nosp,18400244,18400250
3,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400277,18400278,18400279,TAT,92,nosp,18400292,18400298
4,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400283,18400284,18400285,TAC,94,nosp,18400298,18400304


In [12]:
genpos_f.loc[:,'reg_seq'] = genpos_f.apply(
            lambda x: str(fa[x['chr']].seq[x['reg_from'] - 1: x['reg_to']]).upper(),
            axis=1)

In [13]:
genpos_f.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq
0,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,nosp,18400034,18400040,GGCTATA
1,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400037,18400038,18400039,TAT,12,nosp,18400052,18400058,AACAAAT
2,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400229,18400230,18400231,TAC,76,nosp,18400244,18400250,ACAATCG
3,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400277,18400278,18400279,TAT,92,nosp,18400292,18400298,CCAGTTA
4,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400283,18400284,18400285,TAC,94,nosp,18400298,18400304,AACAAAT


In [14]:
genpos_f.shape

(206225, 15)

In [15]:
pam_f = []
for index, data in genpos_f.iterrows():
    for i in re.finditer('(?=GG)',data['reg_seq']):
        NGG_pos = i.span()[1]
        pam_f.append([data['chr'],data['strand'],data['gene_name'],
                      data['transcript_id'],data['pro_id'],data['base1'],
                      data['base2'],data['base3'],data['aa_genome_seq'],
                      data['aa_position'],data['type'],data['reg_from'],
                      data['reg_to'],data['reg_seq'],NGG_pos])

In [16]:
pam_f = DataFrame(pam_f,columns = ['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id',
       'base1', 'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type',
       'reg_from', 'reg_to', 'reg_seq', 'NGG_pos'])

In [17]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,nosp,18400034,18400040,GGCTATA,0
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,0
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,1
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400481,18400482,18400483,TAC,160,nosp,18400496,18400502,AGGAGAG,1
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400739,18400740,18400741,TAT,246,nosp,18400754,18400760,GATCTGG,5


In [18]:
pam_f.shape

(91137, 15)

In [19]:
pam_f.loc[:,'distance'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - pam_f['base2']

In [20]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,nosp,18400034,18400040,GGCTATA,0,13
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,0,13
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,1,14
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400481,18400482,18400483,TAC,160,nosp,18400496,18400502,AGGAGAG,1,14
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400739,18400740,18400741,TAT,246,nosp,18400754,18400760,GATCTGG,5,18


In [21]:
pam_f.loc[pam_f['distance'].isin([14,15,16]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 19
pam_f.loc[pam_f['distance'].isin([14,15,16]),'sg_to'] = pam_f['sg_from'] + 19 - 1

pam_f.loc[pam_f['distance'].isin([13,17]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 20
pam_f.loc[pam_f['distance'].isin([13,17]),'sg_to'] = pam_f['sg_from'] + 20 - 1

pam_f.loc[pam_f['distance'].isin([18]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 21
pam_f.loc[pam_f['distance'].isin([18]),'sg_to'] = pam_f['sg_from'] + 21 - 1

pam_f['sg_from'] = pam_f['sg_from'].map(lambda x:('%.0f')%x).astype(int)
pam_f['sg_to'] = pam_f['sg_to'].map(lambda x:('%.0f')%x).astype(int)

In [22]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,nosp,18400034,18400040,GGCTATA,0,13,18400013,18400032
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,0,13,18400352,18400371
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,1,14,18400354,18400372
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400481,18400482,18400483,TAC,160,nosp,18400496,18400502,AGGAGAG,1,14,18400477,18400495
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400739,18400740,18400741,TAT,246,nosp,18400754,18400760,GATCTGG,5,18,18400737,18400757


In [23]:
pam_f.loc[:,'sg_seq'] = pam_f.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_from'] - 1: x['sg_to']]).upper(),
            axis=1)

In [24]:
pam_f.loc[:,'pam'] = pam_f.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] : x['sg_to'] + 3]).upper(),
            axis=1)

In [25]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,nosp,18400034,18400040,GGCTATA,0,13,18400013,18400032,GAAGCATATTTTGAAAGAAT,TGG
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,0,13,18400352,18400371,AGGAATTACATTGTCGATGC,TGG
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,18400373,18400379,GGGTCTG,1,14,18400354,18400372,GAATTACATTGTCGATGCT,GGG
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400481,18400482,18400483,TAC,160,nosp,18400496,18400502,AGGAGAG,1,14,18400477,18400495,CTGGTACCTGGACCAAATC,AGG
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400739,18400740,18400741,TAT,246,nosp,18400754,18400760,GATCTGG,5,18,18400737,18400757,ATTATAAAGACAATACAGATC,TGG


In [26]:
pam_f.shape

(91137, 20)

In [27]:
pam_f.loc[:,'aa_id'] = pam_f['chr'].str.cat(pam_f['base2'].astype(str),sep='-')
pam_f.loc[:,'sg_id'] = pam_f['aa_id'].str.cat(pam_f['sg_seq'].astype(str),sep='-')

In [28]:
pam_f['aa_id'].unique().shape

(46173,)

In [29]:
pam_f['sg_id'].unique().shape

(61522,)

In [30]:
pam_f['sg_seq'].unique().shape

(59359,)

In [31]:
#reverse strand sgRNA

In [32]:
genpos_r.loc[:,'reg_from'] = genpos_r['base2'] - 20 
genpos_r.loc[:,'reg_to'] = genpos_r['base2'] - 14

In [33]:
genpos_r.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to
205609,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113537,9113538,9113539,TAT,31,nosp,9113518,9113524
205610,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113471,9113472,9113473,TAC,53,nosp,9113452,9113458
205611,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9112393,9112394,9112395,TAC,138,nosp,9112374,9112380
205612,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9110320,9110321,9110322,TAC,166,nosp,9110301,9110307
205613,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9109931,9109932,9109933,TAC,203,nosp,9109912,9109918


In [34]:
genpos_r.loc[:,'reg_seq'] = genpos_r.apply(
            lambda x: str(fa[x['chr']].seq[x['reg_from'] - 1: x['reg_to']].reverse_complement()).upper(),
            axis=1)

In [35]:
genpos_r.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq
205609,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113537,9113538,9113539,TAT,31,nosp,9113518,9113524,CCCTCCC
205610,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113471,9113472,9113473,TAC,53,nosp,9113452,9113458,GTGACTG
205611,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9112393,9112394,9112395,TAC,138,nosp,9112374,9112380,ACAGGTA
205612,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9110320,9110321,9110322,TAC,166,nosp,9110301,9110307,AACATGA
205613,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9109931,9109932,9109933,TAC,203,nosp,9109912,9109918,CAGAAGA


In [36]:
pam_r = []
for index, data in genpos_r.iterrows():
    for i in re.finditer('(?=GG)',data['reg_seq']):
        NGG_pos = i.span()[1]
        pam_r.append([data['chr'],data['strand'],data['gene_name'],
                      data['transcript_id'],data['pro_id'],data['base1'],
                      data['base2'],data['base3'],data['aa_genome_seq'],
                      data['aa_position'],data['type'],data['reg_from'],
                      data['reg_to'],data['reg_seq'],NGG_pos])

In [37]:
pam_r = DataFrame(pam_r,columns = ['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id',
       'base1', 'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type',
       'reg_from', 'reg_to', 'reg_seq', 'NGG_pos'])

In [38]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9112393,9112394,9112395,TAC,138,nosp,9112374,9112380,ACAGGTA,3
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9107635,9107636,9107637,TAT,256,nosp,9107616,9107622,CCTGGAC,3
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9106582,9106583,9106584,TAT,301,nosp,9106563,9106569,ACCAAGG,5
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104326,9104327,9104328,TAT,393,nosp,9104307,9104313,ACCACGG,5
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104323,9104324,9104325,TAC,394,nosp,9104304,9104310,ACGGATG,2


In [39]:
pam_r.shape

(88782, 15)

In [40]:
pam_r.loc[:,'distance'] = pam_r['base2'] - (pam_r['reg_to'] - pam_r['NGG_pos'] + 1)

In [41]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9112393,9112394,9112395,TAC,138,nosp,9112374,9112380,ACAGGTA,3,16
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9107635,9107636,9107637,TAT,256,nosp,9107616,9107622,CCTGGAC,3,16
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9106582,9106583,9106584,TAT,301,nosp,9106563,9106569,ACCAAGG,5,18
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104326,9104327,9104328,TAT,393,nosp,9104307,9104313,ACCACGG,5,18
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104323,9104324,9104325,TAC,394,nosp,9104304,9104310,ACGGATG,2,15


In [42]:
pam_r.loc[pam_r['distance'].isin([14,15,16]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 19
pam_r.loc[pam_r['distance'].isin([14,15,16]),'sg_to'] = pam_r['sg_from'] - 19 + 1

pam_r.loc[pam_r['distance'].isin([13,17]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 20
pam_r.loc[pam_r['distance'].isin([13,17]),'sg_to'] = pam_r['sg_from'] - 20 + 1

pam_r.loc[pam_r['distance'].isin([18]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 21
pam_r.loc[pam_r['distance'].isin([18]),'sg_to'] = pam_r['sg_from'] - 21 + 1

pam_r['sg_from'] = pam_r['sg_from'].map(lambda x:('%.0f')%x).astype(int)
pam_r['sg_to'] = pam_r['sg_to'].map(lambda x:('%.0f')%x).astype(int)

In [43]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9112393,9112394,9112395,TAC,138,nosp,9112374,9112380,ACAGGTA,3,16,9112397,9112379
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9107635,9107636,9107637,TAT,256,nosp,9107616,9107622,CCTGGAC,3,16,9107639,9107621
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9106582,9106583,9106584,TAT,301,nosp,9106563,9106569,ACCAAGG,5,18,9106586,9106566
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104326,9104327,9104328,TAT,393,nosp,9104307,9104313,ACCACGG,5,18,9104330,9104310
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104323,9104324,9104325,TAC,394,nosp,9104304,9104310,ACGGATG,2,15,9104328,9104310


In [44]:
pam_r.loc[:,'sg_seq'] = pam_r.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] - 1: x['sg_from']].reverse_complement()).upper(),
            axis=1)

In [45]:
pam_r.loc[:,'pam'] = pam_r.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] - 4 : x['sg_to'] - 1].reverse_complement()).upper(),
            axis=1)

In [46]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9112393,9112394,9112395,TAC,138,nosp,9112374,9112380,ACAGGTA,3,16,9112397,9112379,TCTACAAACCAGGGCAGAC,AGG
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9107635,9107636,9107637,TAT,256,nosp,9107616,9107622,CCTGGAC,3,16,9107639,9107621,CATATGGGAAGCCTGTCCC,TGG
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9106582,9106583,9106584,TAT,301,nosp,9106563,9106569,ACCAAGG,5,18,9106586,9106566,TCTATCAGCAAGTAAAAACCA,AGG
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104326,9104327,9104328,TAT,393,nosp,9104307,9104313,ACCACGG,5,18,9104330,9104310,ACTATTACTCCAATGCTACCA,CGG
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9104323,9104324,9104325,TAC,394,nosp,9104304,9104310,ACGGATG,2,15,9104328,9104310,TATTACTCCAATGCTACCA,CGG


In [47]:
pam_r.shape

(88782, 20)

In [48]:
pam_r.loc[:,'aa_id'] = pam_r['chr'].str.cat(pam_r['base2'].astype(str),sep='-')
pam_r.loc[:,'sg_id'] = pam_r['aa_id'].str.cat(pam_r['sg_seq'].astype(str),sep='-')

In [49]:
pam_r['aa_id'].unique().shape

(46489,)

In [50]:
pam_r['sg_id'].unique().shape

(61922,)

In [51]:
pam_r['sg_seq'].unique().shape

(60050,)

In [52]:
#sgRNA filter with 0.2 < GC content < 0.8 & remove TTTT

In [53]:
sg = pam_f.append(pam_r)

In [54]:
sg.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,...,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam,aa_id,sg_id
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400019,18400020,18400021,TAT,6,...,18400040,GGCTATA,0,13,18400013,18400032,GAAGCATATTTTGAAAGAAT,TGG,chr8-18400020,chr8-18400020-GAAGCATATTTTGAAAGAAT
1,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,...,18400379,GGGTCTG,0,13,18400352,18400371,AGGAATTACATTGTCGATGC,TGG,chr8-18400359,chr8-18400359-AGGAATTACATTGTCGATGC
2,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,...,18400379,GGGTCTG,1,14,18400354,18400372,GAATTACATTGTCGATGCT,GGG,chr8-18400359,chr8-18400359-GAATTACATTGTCGATGCT
3,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400481,18400482,18400483,TAC,160,...,18400502,AGGAGAG,1,14,18400477,18400495,CTGGTACCTGGACCAAATC,AGG,chr8-18400482,chr8-18400482-CTGGTACCTGGACCAAATC
4,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400739,18400740,18400741,TAT,246,...,18400760,GATCTGG,5,18,18400737,18400757,ATTATAAAGACAATACAGATC,TGG,chr8-18400740,chr8-18400740-ATTATAAAGACAATACAGATC


In [55]:
sg.loc[:,'GC_content'] = (sg['sg_seq'].str.count('G') + sg['sg_seq'].str.count('C')) / sg['sg_seq'].str.len()

In [56]:
sg_gc = sg[(sg['GC_content'] >= 0.2) & (sg['GC_content'] <= 0.8)]

In [57]:
sg_gc_4t = sg_gc.append(
    sg_gc[sg_gc['sg_seq'].str.contains('TTTT')]).drop_duplicates(keep=False)

In [58]:
sg_gc_4t.columns

Index(['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id', 'base1',
       'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type', 'reg_from',
       'reg_to', 'reg_seq', 'NGG_pos', 'distance', 'sg_from', 'sg_to',
       'sg_seq', 'pam', 'aa_id', 'sg_id', 'GC_content'],
      dtype='object')

In [59]:
sg_gc_4t = sg_gc_4t[['sg_id','aa_id','chr','strand','gene_name','transcript_id',
                    'pro_id','base1','base2','base3','aa_genome_seq','aa_position',
                    'type','sg_seq','sg_from','sg_to','pam','GC_content']]

In [60]:
sg_gc_4t.head()

,sg_id,aa_id,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,sg_seq,sg_from,sg_to,pam,GC_content
1,chr8-18400359-AGGAATTACATTGTCGATGC,chr8-18400359,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,AGGAATTACATTGTCGATGC,18400352,18400371,TGG,0.400000
2,chr8-18400359-GAATTACATTGTCGATGCT,chr8-18400359,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400358,18400359,18400360,TAC,119,nosp,GAATTACATTGTCGATGCT,18400354,18400372,GGG,0.368421
3,chr8-18400482-CTGGTACCTGGACCAAATC,chr8-18400482,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400481,18400482,18400483,TAC,160,nosp,CTGGTACCTGGACCAAATC,18400477,18400495,AGG,0.526316
4,chr8-18400740-ATTATAAAGACAATACAGATC,chr8-18400740,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400739,18400740,18400741,TAT,246,nosp,ATTATAAAGACAATACAGATC,18400737,18400757,TGG,0.238095
5,chr1-75734837-AGTATTTGGGGAGAATGACTG,chr1-75734837,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75734836,75734837,75734838,TAT,145,nosp,AGTATTTGGGGAGAATGACTG,75734834,75734854,AGG,0.428571


In [61]:
sg_gc_4t['sg_id'].unique().shape

(117967,)

In [62]:
sg_gc_4t['aa_id'].unique().shape

(88529,)

In [63]:
sg_gc_4t['sg_seq'].unique().shape

(112787,)

In [64]:
sg_gc_4t.to_csv('../data/tyr/sg_tyr.txt',sep='\t',index=False)